In [1]:
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [3]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
rental = pd.read_sql_query('SELECT * FROM rental', engine)
rental.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [23]:
rental_may = rental[rental.rental_date.dt.month == 5]
count_rentals_by_customer = rental_may.groupby('customer_id')['rental_id'].count()
count_rentals_by_customer

customer_id
1      2
2      1
3      2
5      3
6      3
      ..
594    4
595    1
596    6
597    2
599    1
Name: rental_id, Length: 520, dtype: int64

In [24]:
rental_june = rental[rental.rental_date.dt.month == 6]
count_rentals_by_customer = rental_june.groupby('customer_id')['rental_id'].count()
count_rentals_by_customer

customer_id
1      7
2      1
3      4
4      6
5      5
      ..
595    2
596    2
597    3
598    1
599    4
Name: rental_id, Length: 590, dtype: int64

In [25]:
query = 'SELECT customer_id, COUNT(rental_id) count_rental\
        FROM rentals_may \
        GROUP BY customer_id \
        ORDER BY 2 DESC;'

data_may = pd.read_sql_query(query, engine)
data_may.head()

,customer_id,count_rental
0,197,8
1,506,7
2,109,7
3,269,6
4,239,6


In [26]:
query = 'SELECT customer_id, COUNT(rental_id) count_rental\
        FROM rentals_june \
        GROUP BY customer_id \
        ORDER BY 2 DESC;'

data_june = pd.read_sql_query(query, engine)
data_june.head()

,customer_id,count_rental
0,31,11
1,454,10
2,329,9
3,295,9
4,561,9


In [109]:
def most_borrows_books(i):
    if i in data_may['customer_id'].tolist() and i not in data_june['customer_id'].tolist():
        return 'Customer didn\'t borrow books in June'
    elif i in data_june['customer_id'].tolist() and i not in data_may['customer_id'].tolist():
        return 'Customer didn\'t borrow books in May'
    elif i not in data_june['customer_id'].tolist() and i not in data_may['customer_id'].tolist():
        return 'Customer doesn\'t exist'
    customer_may = data_may[data_may['customer_id'] == i].count_rental.values[0]
    customer_june = data_june[data_june['customer_id'] == i].count_rental.values[0]

    if customer_may > customer_june:
        return 'Customer borrowed {} books in May'.format(customer_may - customer_june)
    elif customer_may < customer_june:
        return 'Customer borrowed {} books in June'.format(customer_june - customer_may)
    else:
        return 'Customer borrowed the same number of books, {}, in May and June'.format(customer_june)

most_borrows_books(454)

'Customer borrowed 9 books in June'

# Option 2 simpler!

In [169]:
df3 = pd.merge(data_may, data_june, on= 'customer_id')
df3.head()

,customer_id,count_rental_x,count_rental_y
0,197,8,8
1,506,7,5
2,109,7,5
3,269,6,3
4,239,6,5


In [170]:
df3 = df3.set_index('customer_id')
df3

,count_rental_x,count_rental_y
customer_id,,
197,8,8
506,7,5
109,7,5
269,6,3
239,6,5
...,...,...
234,1,8
431,1,7
351,1,3


In [175]:
def books_borrowed(i):
    subs = df3.loc[i,:][0] - df3.loc[i,:][1]
    if subs > 0:
        return 'Customer borrowed {} books in May'.format(subs)
    elif subs < 0:
        return 'Customer borrowed {} books in June'.format(-(subs))
    else:
        return 'Customer borrowed the same books, {}, in both monts'.format(df3.loc[i,:][1])

books_borrowed(10)

'Customer borrowed 4 books in June'